### ¿Qué es Apache Spark?

Apache spark es un framework de programación para el procesamiento de datos distribuidos (computacion en cluster) open source. Spark fue originalmente desarrolado por la Universidad de California Berkeley quien posteriomente donó el codigo del proyecto a la Apache Software Foundation, quien desde entonces se encarga de su mantenimiento.

Spark proporciona APIs para ser empleado en conjunto con lenguajes de programación como Java, Python, Scala y R. Además, cuenta con herramientas para la realización de tareas de alto nivel como Spark SQL (para el procesamiento de datos estructurados basado en SQL), MLlib (para la implementación de modelos de aprendizaje máquina), GraphX (para el procesamiento de grafos), etc.

### ¿Qué es PySpark?

PySpark es una herramienta que nos deja usar Spark encima de Python. Permitiendonos combinar el proceso de datos distribuidos de Spark con la simplicidad de Python para el análisis de conjuntos masivos de datos (big data).

In [3]:
!pip install pyspark
!pip install -U -q PyDrive
# !apt install openjdk-8-jdk-headless -qq

     ---------------------------------------- 0.0/455.4 MB ? eta -:--:--
     --------------------------------------- 1.0/455.4 MB 14.6 MB/s eta 0:00:32
     --------------------------------------- 4.5/455.4 MB 16.0 MB/s eta 0:00:29
      -------------------------------------- 7.9/455.4 MB 15.8 MB/s eta 0:00:29
      ------------------------------------- 11.3/455.4 MB 15.7 MB/s eta 0:00:29
     - ------------------------------------ 14.2/455.4 MB 15.3 MB/s eta 0:00:29
     - ------------------------------------ 17.3/455.4 MB 15.4 MB/s eta 0:00:29
     - ------------------------------------ 20.7/455.4 MB 15.5 MB/s eta 0:00:28
     -- ----------------------------------- 24.1/455.4 MB 15.5 MB/s eta 0:00:28
     -- ----------------------------------- 27.3/455.4 MB 15.5 MB/s eta 0:00:28
     -- ----------------------------------- 30.7/455.4 MB 15.5 MB/s eta 0:00:28
     -- ----------------------------------- 34.1/455.4 MB 15.6 MB/s eta 0:00:27
     --- ---------------------------------- 37.


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [ ]:
import pyspark
# Carga ufnciones extra
from pyspark.sql.functions import * 
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('primeros_pasos').getOrCreate()

### RDDs

La forma más elemental de trabajar con datos en PySpark es haciendo uso de un RDD (Resilient Distributed Dataset), el cual nos da la capacidad de manejar los datos y realizar cálculos con ellos de forma distribuida en varios nodos de un cluster.

Para realizar los ejemplos de este notebook haremos uso de una base de datos que cuenta con datos de los jugadores del juego FIFA20, dichos datos fueron tomados este concurso de kaggle donde podrán encontrar una descripción detalalda de cada campo de la base datos. https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset/version/1?select=players_20.csv

Podemos crear un RDD a partir de un archivo CSV de la siguiente manera

In [4]:
raw_data = spark.sparkContext.textFile('../data/players_20.csv')

In [5]:
raw_data

../data/players_20.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

Una de las desventajas de los RDDs es que solo proveen a los datos de la estructura minima para poder trabajar con ellos. Imprimamos los primeros dos elementos del RDD para ver en que condicion se encuentran nuestros datos

In [6]:
for row in raw_data.take(2):
    print(row)
    print('\n')

sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defe

Como podemos ver, los elementos del RDD son solo los renglones del archivo CSV sin formato. Siendo el primer elemento los nombres de los campos de la base de datos, el mantener a los nombres de los campos como valores dentro del RDD puede derivar en problemas más adelante, así que por el momento los removeremos para quedarnos solamente con los valores

In [7]:
# Guarda el primer elemento 
columns = raw_data.take(1)[0]
# Guarda en todos los elementos de datos_crudos menos el primero correspondiente a las columnas
raw_data = raw_data.filter(lambda row : row != columns)

In [8]:
columns

'sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,def

In [9]:
raw_data

PythonRDD[4] at RDD at PythonRDD.scala:53

Ahora podemos darle un poco más de estructura a nuestros datos. Primero separemos los campos de cada renglon, sabiendo que estan separados por comas, y quedandonos con solo un subconjunto de dichos campos, digamos... su nombre, edad, estatura, peso, nacionalidad, equipo y su ranking. Para esto definamos la siguiente función

In [10]:
def separar_campos(row):
    separated_row = row.split(',')
    # Extrae los campos que queremos
    nombre = str(separated_row[2])
    edad = int(separated_row[4])
    estatura = int(separated_row[6])
    peso = int(separated_row[7])
    nacionalidad = str(separated_row[8])
    equipo = str(separated_row[9])
    rank = int(separated_row[10])

    return (nombre, edad, estatura, peso, nacionalidad, equipo, rank)



aplicamos dicha función a cada renglon



In [11]:
rdd_fifa = raw_data.map(separar_campos)

# Imprimimos una muestra de los datos
for row in rdd_fifa.take(5):
    print(row)
print('\n')

('L. Messi', 32, 170, 72, 'Argentina', 'FC Barcelona', 94)
('Cristiano Ronaldo', 34, 187, 83, 'Portugal', 'Juventus', 93)
('Neymar Jr', 27, 175, 68, 'Brazil', 'Paris Saint-Germain', 92)
('J. Oblak', 26, 188, 87, 'Slovenia', 'Atlético Madrid', 91)
('E. Hazard', 28, 175, 74, 'Belgium', 'Real Madrid', 91)




Hasta ahora, hemos realizado diferentes operaciones sobre nuestro RDD para poder darle un formato más tratable a los datos que contiene. Sin embargo, no hemos detallado nada sobre la naturaleza de dichas operaciones. Enfoquemonos ahora en hacer más explícitos estos detalles.

Existen dos operaciones básicas que pueden ser aplicadas a los RDD:

    Acciones: Son operaciones que accionan la realización de cálculos y regresan valores explícitos del RDD.

    Transformaciones: Son operaciones que devuelven otro RDD. Este tipo de operaciones no se ejecutan como tal sino hasta que es llamada una acción. Por esta razón se dice que las transformaciones son de evaluacion peresoza (lazy evaluation).

En el caso de la celda anterior, map() es una transformación que aplica la función dada a los elementos del RDD y take() es la acción que nos devuelve los

primeros elementos.

Veamos más ejemplos. Podríamos querer saber cuantos jugadores existen por país en la base de datos. Para lograr esto primero necesitamos convertir nuestros datos en pares (key, value), donde la keyserá el pais y el value nos indicará que ese registro pertenece a un jugador de dicho país

In [12]:
# Nos quedamos solo con la entrada corresponiente al país
rdd_nacionalidades = rdd_fifa.map(lambda row: (row[4], 1))

In [13]:
rdd_nacionalidades

PythonRDD[6] at RDD at PythonRDD.scala:53

Usando los países como llaves y la transfromación llamada reduceByKey, sumaremos todos los valores para cada país.

In [14]:
rdd_nacionalidades = rdd_nacionalidades.reduceByKey(lambda val_1 , val_2: val_1 + val_2)

In [15]:
rdd_nacionalidades

PythonRDD[11] at RDD at PythonRDD.scala:53

Hasta el momento hemos usado las transformaciones map() y reduceByKey(), para obtener resultados necesitamos efectuar una acción, en este caso usaremos la acción collect(), que nos devuelve todo el contenido del RDD. Esta acción debe usarse con cuidado, en casos donde el RDD contenga una gran cantidad de registros, cargar todo el contenido del RDD puede dejar al sistema sin memoria.

In [16]:
resultados = rdd_nacionalidades.collect()

for row in resultados[:20]:
    print(row)

('Argentina', 886)
('Portugal', 344)
('Brazil', 824)
('Slovenia', 61)
('Belgium', 268)
('Netherlands', 416)
('France', 984)
('Uruguay', 164)
('Poland', 324)
('Denmark', 345)
('Gabon', 16)
('Korea Republic', 322)
('Costa Rica', 30)
('Slovakia', 54)
('Bosnia Herzegovina', 66)
('Scotland', 277)
('Hungary', 35)
('Switzerland', 229)
('Greece', 96)
('Austria', 319)


Aunque los resultados obtenidos son buenos, sería interesante poder ordenar a los paises por la cantidad de jugadores. En la siguiente celda imprimimos los 10 países con más jugadores registrados en el juego de FIFA20

In [17]:
# Pedimos que ordene por total de jugadores
rdd_nacionalidades = rdd_nacionalidades.sortBy(lambda row: row[1], ascending = False)

print ("{:<12} {:<15}".format('PAIS', 'TOTAL JUGADORES'))

for row in rdd_nacionalidades.take(10):
    pais = row[0]
    total_jugadores = row[1]
  
    print ("{:<12} {:<15}".format(pais, total_jugadores))

PAIS         TOTAL JUGADORES
England      1667           
Germany      1216           
Spain        1035           
France       984            
Argentina    886            
Brazil       824            
Italy        732            
Colombia     591            
Japan        453            
Netherlands  416            


Además de las transformaciones y acciones empleadas aquí existen muchas más, sin embargo, los ejemplos expuestos aqui pueden ayudar al lector a familiarizarse con su uso.

Como el lector habrá notado, aunque los RDDs nos proveen de la capacidad de manipular y hacer calculos con datos distribuidos, éstos carecen de las funcionalidades y la estructura que tienen herramientas de un poco más alto nivel, como los DataFrame de Pandas, para el manejo de datos.

En PySpark existe un tipo de objetos, igual llamados DataFrames, los cuales nos ayudan a dotar a nuestros datos con más funcionalidades y estructura, en comparación a los RDDs, al mismo tiempo que seguir trabajando de forma distribuida con ellos. Por esta razon, procederemos a dar una introducción a los DataFrames de PySpark que son los entes con los cuales más trabajaremos a lo largo del curso.

### DataFrames

A diferencia de los RDDs, los DataFrames nos permiten estructurar a una collección de datos distribuidos en forma de tabla (con columnas y filas).

Podemos crear un DataFrame a partir de un archivo CSV de la siguiente manera

In [18]:
df_fifa = spark.read.csv('../data/players_20.csv', header = True, inferSchema = True)

Para efectos de la clase no haremos uso de todos los campos de la base de datos, seleccionemos solo un subconjunto. Para esto usaremos la función select(), la cual devuelve un DataFrame con solo las columnas seleccionadas.

In [19]:
df_fifa = df_fifa.select('short_name','age', 'height_cm', 'weight_kg', 
                         'nationality', 'club', 'value_eur', 'preferred_foot')

Dada la estructrua que el DataFrame da a los datos, podemos hacer referencia a las columnas por su nombre en el archivo CSV.
Para visualizar los datos, utilizamos:

In [20]:
df_fifa.show(15)

+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona| 95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus| 58500000|         Right|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|105500000|         Right|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid| 77500000|         Right|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid| 90000000|         Right|
|     K. De Bruyne| 28|      181|       70|    Belgium|    Manchester City| 90000000|         Right|
|    M. ter Stegen| 27|      187|       85|    Germany|       FC Barcelona| 67500000|      

Si el contenido del DataFrame es lo suficientemente pequeño podemos transformar su contenido a un DataFrame de Pandas.

In [21]:
df_pequeño = df_fifa.limit(10)
df_pandas = df_pequeño.toPandas()
df_pandas.head(5)

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,L. Messi,32,170,72,Argentina,FC Barcelona,95500000,Left
1,Cristiano Ronaldo,34,187,83,Portugal,Juventus,58500000,Right
2,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,105500000,Right
3,J. Oblak,26,188,87,Slovenia,Atlético Madrid,77500000,Right
4,E. Hazard,28,175,74,Belgium,Real Madrid,90000000,Right


In [22]:
df_fifa

DataFrame[short_name: string, age: int, height_cm: int, weight_kg: int, nationality: string, club: string, value_eur: int, preferred_foot: string]

Con Spark podemos hacer lo anterior con una sola línea de código.

In [23]:
df_fifa.limit(10).toPandas().head(5)

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,L. Messi,32,170,72,Argentina,FC Barcelona,95500000,Left
1,Cristiano Ronaldo,34,187,83,Portugal,Juventus,58500000,Right
2,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,105500000,Right
3,J. Oblak,26,188,87,Slovenia,Atlético Madrid,77500000,Right
4,E. Hazard,28,175,74,Belgium,Real Madrid,90000000,Right


In [24]:
df_fifa.limit(10) \
  .toPandas() \
  .head(5)

,short_name,age,height_cm,weight_kg,nationality,club,value_eur,preferred_foot
0,L. Messi,32,170,72,Argentina,FC Barcelona,95500000,Left
1,Cristiano Ronaldo,34,187,83,Portugal,Juventus,58500000,Right
2,Neymar Jr,27,175,68,Brazil,Paris Saint-Germain,105500000,Right
3,J. Oblak,26,188,87,Slovenia,Atlético Madrid,77500000,Right
4,E. Hazard,28,175,74,Belgium,Real Madrid,90000000,Right


Para poder revisar la organización de nuestro DataFrame y el data tpye de cada campo podemos usar lo siguiente

In [26]:
df_fifa.printSchema()

root
 |-- short_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- height_cm: integer (nullable = true)
 |-- weight_kg: integer (nullable = true)
 |-- nationality: string (nullable = true)
 |-- club: string (nullable = true)
 |-- value_eur: integer (nullable = true)
 |-- preferred_foot: string (nullable = true)



### Funciones

#### Renombrar columnas

In [27]:
df_fifa.withColumnRenamed('value_eur','valor_euros').show(5)

+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|valor_euros|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona|   95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus|   58500000|         Right|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|  105500000|         Right|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid|   77500000|         Right|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid|   90000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
only showing top 5 rows



In [28]:
df_ejemplo = df_fifa.withColumnRenamed('value_eur','valor_euros')
df_ejemplo.show(5)

+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|               club|valor_euros|preferred_foot|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona|   95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus|   58500000|         Right|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|  105500000|         Right|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid|   77500000|         Right|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid|   90000000|         Right|
+-----------------+---+---------+---------+-----------+-------------------+-----------+--------------+
only showing top 5 rows



#### Quitar columnas

In [29]:
df_fifa.drop('weight_kg').show(5)

+-----------------+---+---------+-----------+-------------------+---------+--------------+
|       short_name|age|height_cm|nationality|               club|value_eur|preferred_foot|
+-----------------+---+---------+-----------+-------------------+---------+--------------+
|         L. Messi| 32|      170|  Argentina|       FC Barcelona| 95500000|          Left|
|Cristiano Ronaldo| 34|      187|   Portugal|           Juventus| 58500000|         Right|
|        Neymar Jr| 27|      175|     Brazil|Paris Saint-Germain|105500000|         Right|
|         J. Oblak| 26|      188|   Slovenia|    Atlético Madrid| 77500000|         Right|
|        E. Hazard| 28|      175|    Belgium|        Real Madrid| 90000000|         Right|
+-----------------+---+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



#### Ordenar

In [30]:
df_fifa.orderBy('value_eur', ascending = False).show(5)

+------------+---+---------+---------+-----------+-------------------+---------+--------------+
|  short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|
+------------+---+---------+---------+-----------+-------------------+---------+--------------+
|   Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|105500000|         Right|
|    L. Messi| 32|      170|       72|  Argentina|       FC Barcelona| 95500000|          Left|
|   K. Mbappé| 20|      178|       73|     France|Paris Saint-Germain| 93500000|         Right|
|   E. Hazard| 28|      175|       74|    Belgium|        Real Madrid| 90000000|         Right|
|K. De Bruyne| 28|      181|       70|    Belgium|    Manchester City| 90000000|         Right|
+------------+---+---------+---------+-----------+-------------------+---------+--------------+
only showing top 5 rows



#### Filtrado con where

In [31]:
df_fifa.where(df_ejemplo.age > 30).show(5)

+-----------------+---+---------+---------+-----------+---------------+---------+--------------+
|       short_name|age|height_cm|weight_kg|nationality|           club|value_eur|preferred_foot|
+-----------------+---+---------+---------+-----------+---------------+---------+--------------+
|         L. Messi| 32|      170|       72|  Argentina|   FC Barcelona| 95500000|          Left|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|       Juventus| 58500000|         Right|
|        L. Modrić| 33|      172|       66|    Croatia|    Real Madrid| 45000000|         Right|
|     G. Chiellini| 34|      187|       85|      Italy|       Juventus| 24500000|          Left|
|        S. Agüero| 31|      173|       70|  Argentina|Manchester City| 60000000|         Right|
+-----------------+---+---------+---------+-----------+---------------+---------+--------------+
only showing top 5 rows



tal vez quisieramos solo ver la informacion de los jugadores del Liverpool que tuviera mas de 30 años de edad

In [32]:
df_fifa.where((df_ejemplo.age > 30) & (df_ejemplo.club == 'Liverpool')).show(5)

+-----------+---+---------+---------+-----------+---------+---------+--------------+
| short_name|age|height_cm|weight_kg|nationality|     club|value_eur|preferred_foot|
+-----------+---+---------+---------+-----------+---------+---------+--------------+
|  J. Milner| 33|      175|       70|    England|Liverpool| 10500000|         Right|
| A. Lallana| 31|      178|       73|    England|Liverpool| 11000000|         Right|
|     Adrián| 32|      190|       80|      Spain|Liverpool|  4700000|         Right|
|A. Lonergan| 35|      192|       87|    England|Liverpool|   150000|          Left|
+-----------+---+---------+---------+-----------+---------+---------+--------------+



#### Funciones de agregación

Supongamos que queremos saber el costo total en euros de un equipo, lograríamos dicho objetivo sumando el valor de los jugadores de cada equipo. Esto lo podemos lograr usando la funcion groupBy() en conjunto con algunas funciones de agregación de la siguiente manera

In [33]:
df_fifa.groupBy('Club').sum('value_eur').show(5)

+--------------------+--------------+
|                Club|sum(value_eur)|
+--------------------+--------------+
|       Côte d'Ivoire|             0|
|          Göztepe SK|      43065000|
|CD Everton de Viñ...|      18075000|
|     Shonan Bellmare|       1410000|
|        Salford City|       7905000|
+--------------------+--------------+
only showing top 5 rows



si quisieramos saber cuales son los 5 equipos mas caros hariamos lo siguiente

In [34]:
df_fifa.groupBy('club') \
  .sum('value_eur') \
  .orderBy('sum(value_eur)', ascending = False) \
  .show(5)

+---------------+--------------+
|           club|sum(value_eur)|
+---------------+--------------+
|    Real Madrid|     897850000|
|   FC Barcelona|     869300000|
|Manchester City|     845745000|
|       Juventus|     735475000|
|      Liverpool|     693265000|
+---------------+--------------+
only showing top 5 rows



Si quisieramos usar más de una función de agregación tendriamos que hacer lo siguiente

#### Crear nuevas columnas

Podemos crear columnas a partir de los valores de otras, por ejemplo, podríamos querer obtener el valor de los jugadores en pesos.

In [35]:
df_fifa.withColumn('valor_pesos', df_fifa.value_eur * 25.2).show(5)

+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+-----------+
|       short_name|age|height_cm|weight_kg|nationality|               club|value_eur|preferred_foot|valor_pesos|
+-----------------+---+---------+---------+-----------+-------------------+---------+--------------+-----------+
|         L. Messi| 32|      170|       72|  Argentina|       FC Barcelona| 95500000|          Left|   2.4066E9|
|Cristiano Ronaldo| 34|      187|       83|   Portugal|           Juventus| 58500000|         Right|   1.4742E9|
|        Neymar Jr| 27|      175|       68|     Brazil|Paris Saint-Germain|105500000|         Right|   2.6586E9|
|         J. Oblak| 26|      188|       87|   Slovenia|    Atlético Madrid| 77500000|         Right|    1.953E9|
|        E. Hazard| 28|      175|       74|    Belgium|        Real Madrid| 90000000|         Right|    2.268E9|
+-----------------+---+---------+---------+-----------+-------------------+---------+-----------

### Consultas SQL

Quien esté familiarizado con SQL habrá notado que existe cierta similaridad entre las funciones de los DataFrames de PySpark con algunas sentencias de SQL. Este paralelismo llega a tal grado que podemos reemplazar la concatenación de varias funciones con una simple consulta SQL. Por ejemplo, si quisieramos consultar de nuevo cuales son los 5 equipos más caros, solo que esta vez haciendo una consulta SQL, haríamos los siguiente

In [34]:
# Le dice a spark que guarde este DataFrame como una tabla SQL
df_fifa.registerTempTable('FIFA20')

consulta = '''
  SELECT club, SUM(value_eur) AS valor_total 
  FROM FIFA20
  GROUP BY club
  ORDER BY valor_total DESC
  '''

spark.sql(consulta).show(10)

/home/vania/jupyter/jupyterenv/lib/python3.8/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(
24/02/24 09:57:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+-----------+
|               club|valor_total|
+-------------------+-----------+
|        Real Madrid|  897850000|
|       FC Barcelona|  869300000|
|    Manchester City|  845745000|
|           Juventus|  735475000|
|          Liverpool|  693265000|
|  FC Bayern München|  688775000|
|Paris Saint-Germain|  687550000|
|  Tottenham Hotspur|  649850000|
|    Atlético Madrid|  590375000|
|  Borussia Dortmund|  532325000|
+-------------------+-----------+
only showing top 10 rows



Basado en los apuntes del Dr. Gibran Fuentes